In [1]:
# Import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd

In [2]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\makin\.wdm\drivers\chromedriver\win32\91.0.4472.19\chromedriver.exe] found in cache


In [3]:
url_news="https://redplanetscience.com/"
browser.visit(url_news)
html = browser.html

In [4]:
# Create a Beautiful Soup object
soup = bs(html, 'html.parser')
soup.title.text

'News - Mars Exploration Program'

# Extracting Latest News Title and Paragraph

In [5]:
# extract latest title
titles = soup.find("div", class_="content_title").text
titles
# extract paragraph
para = soup.find("div", class_="article_teaser_body").text
para

'Headed to the Red Planet with the Perseverance rover, the pioneering helicopter is powered up for the first time in interplanetary space as part of a systems check.'

# Image Scrapping

In [6]:
# Open browser 
url_image="https://spaceimages-mars.com/"
browser.visit(url_image)
html = browser.html
soup = bs(html, 'html.parser')

In [7]:
head = soup.find('div', class_="floating_text_area")
image_url = head.find('a')['href']

featured_image_url = url_image + image_url
featured_image_url


'https://spaceimages-mars.com/image/featured/mars1.jpg'

# Scraping Table by Pandas

In [8]:
table_url="https://galaxyfacts-mars.com/"

table = pd.read_html(table_url)
df = table[0]
df.columns=['Desciptions','Mars', 'Earth']
df


,Desciptions,Mars,Earth
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [9]:
df.to_html('table.html', index=False, classes=['table', 'table-striped'])

# Scrapping Mars Hemisphere Titles and Images

In [10]:
# Visit url
hemis_url="https://marshemispheres.com/"
browser.visit(hemis_url)
html = browser.html

# Creating bs object
soup = bs(html, 'html.parser')
soup.title.text

# Scrapping Mars Hemisphere Titles and Images
temp = soup.find_all('div', class_="description")

hemisphere_title = []
for x in temp:
    hemisphere_title.append(x.find('h3').text)

# print(hemisphere_title)

image_full_url = []
for hemis in hemisphere_title:

    html = browser.html
    soup = bs(html, 'html.parser')

    browser.links.find_by_partial_text(f'{hemis}').click()

    html = browser.html
    soup = bs(html, 'html.parser')

    tt = soup.find('img', class_="wide-image")
    image_full_url.append(tt['src'])

    browser.links.find_by_partial_text('Back').click()

browser.quit()

hemis_image_url = [hemis_url + url for url in image_full_url]
hemisphere_image_urls = []
for i in range(len(hemisphere_title)):
    hemisphere_image_urls.append(
        {"title": hemisphere_title[i],
        "img_url" : hemis_image_url[i]
        }
    )
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [11]:
# Close the browser
browser.quit()